# DATA PRERPARATION FOR THE MODEL AND ANALYSIS

In [49]:
import pandas as pd
import pandas
pandas.set_option('display.max_columns', None)
import os

### Get the data containing the referee and concatinate it into one csv file for each leag

In [50]:
#current directory
current_dir = os.getcwd()

for folder in os.listdir(current_dir):
    if os.path.isdir(os.path.join(current_dir, folder)):
        dataframes = []
        folder_path = os.path.join(current_dir, folder)
        
        # List CSV files in the current folder
        csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
        
        # Read CSV files into dataframes and append to the list
        for file in csv_files:
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            dataframes.append(df)

        # Concatenate all dataframes in the list
        concatenated = pd.concat(dataframes, ignore_index=True)
        concatenated.to_csv(folder+'_Referee.csv', index=False)


### Join then with our original data

In [51]:
csv_files = [ 'Fixtures_ENG_Premier_League_elo.csv' , 'ENG_Referee.csv',
              'Fixtures_FRA_Ligue_1_elo.csv'        , 'FR_Referee.csv' ,
              'Fixtures_GER_Bundesliga_elo.csv'     , 'GER_Referee.csv',
              'Fixtures_ITA_Serie_A_elo.csv'        , 'ITA_Referee.csv',
              'Fixtures_ESP_La_Liga_elo.csv'        , 'ESP_Referee.csv']

In [52]:
#Drop uncecary columns
def dropColumns(df):
    keep = ['Date', 'HomeTeam', 'AwayTeam', 'Referee']  
    drop = [col for col in df.columns if col not in keep]
    df.drop(columns=drop, inplace=True)

In [53]:
#Convert the date format to yyyy-mm-dd
def convDate(df):
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%y').dt.strftime('%Y-%m-%d')

In [54]:
# Join the original df with the new df containing the referee
def JoinData(data, ref):
    data['fixture_date'] = pd.to_datetime(data['fixture_date'])
    ref['Date'] = pd.to_datetime(ref['Date'])
    merged = pd.merge(data, ref,  how='left', left_on=['fixture_date', 'teams_home_name', 'teams_away_name'], right_on=['Date', 'HomeTeam','AwayTeam'])
    return merged

In [55]:
# Iterate and copy the referee name to the NaN values in the fixture_referee 
def copyReferee(row):
    if pd.isnull(row['fixture_referee']):
        row['fixture_referee'] = row['Referee']
    return row

In [56]:
#Drop the columns used for joining
def DropAfterJoin(df):
    df.drop(columns=['Date','HomeTeam',	'AwayTeam',	'Referee', 'venue_id', 'venue_city'], inplace=True)

In [57]:
# Remove the country parth from the fixture_referee
def removeCountry(location):
    if ',' in location:
        split_location = location.split(',', 1) 
        return split_location[0]
    return location

In [58]:
# Transform the referee name to one format
def transform_name(name):
    split_name = name.split() 
    if len(split_name) >= 2:
        return split_name[0][0] + ' ' + split_name[-1]
    return name

In [62]:
#Add the referee to the original data
def addReff(data, ref):
    dropColumns(ref)
    convDate(ref)
    joined = JoinData(data,ref)
    joined = joined.apply(copyReferee, axis=1)
    DropAfterJoin(joined)
    joined['fixture_referee'] = joined['fixture_referee'].apply(removeCountry)
    joined['fixture_referee'] = joined['fixture_referee'].apply(transform_name)
    return joined

In [68]:
data = pd.read_csv('Fixtures_ENG_Premier_League_elo.csv')
ref = pd.read_csv('ENG_Referee.csv')


print(data.columns)
print('REFEREE')
print(ref.columns)

df = addReff(data,ref)


Index(['fixture_id', 'fixture_referee', 'fixture_date', 'venue_id',
       'venue_name', 'venue_city', 'status_long', 'status_short',
       'status_elapsed', 'league_id', 'league_name', 'league_country',
       'league_logo', 'league_flag', 'league_season', 'league_round',
       'teams_home_id', 'teams_home_name', 'teams_home_logo',
       'teams_home_winner', 'teams_away_id', 'teams_away_name',
       'teams_away_logo', 'teams_away_winner', 'goals_home', 'goals_away',
       'score_halftime_home', 'score_halftime_away', 'score_fulltime_home',
       'score_fulltime_away', 'home_rank', 'home_elo', 'away_rank',
       'away_elo'],
      dtype='object')
REFEREE
Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD',
       'BWA', 'GBH', 'GBD', 'GBA', 'IWH', 'IWD', 'IWA', 'LBH', 'LBD', 'LBA',
       'SBH', 'SBD', 

In [67]:
for i in range(0, len(csv_files), 2):
  data = pd.read_csv(csv_files[i])
  ref = pd.read_csv(csv_files[i + 1])
  df = addReff(data,ref)
  df.to_csv('C:\\Users\\Imad\\Desktop\\S3\\BigData\\YaFaa-Flask\\YAFAA_Fixtures_Elo\\Refer_'+str(csv_files[i]))

KeyError: 'Referee'